# Notebook 1 — Market Sentiment & Trader Behavior Analysis

**Candidate:** Yamini Priya

**Objective:** Explore how trader behaviour (profitability, risk, volume, leverage) aligns with market sentiment (Fear vs Greed). This notebook is designed to run in Google Colab.  

## 1. Setup and imports
Install required packages (if running in Colab) and import libraries.

In [ ]:

# If running in Colab uncomment and run the following:
# !pip install -q pandas matplotlib seaborn

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Ensure output folders exist
os.makedirs('/mnt/data/outputs', exist_ok=True)
os.makedirs('/mnt/data/csv_files', exist_ok=True)


## 2. Load datasets
Upload or mount your files to `/mnt/data/` in this environment and run the cell below. The example uses the provided CSVs.

In [ ]:

# Paths (adjust if needed)
hist_path = '/mnt/data/historical_data.csv'
fg_path = '/mnt/data/fear_greed_index.csv'

# Load CSVs
hist = pd.read_csv(hist_path)
fg = pd.read_csv(fg_path)

print('Historical rows:', len(hist))
print('Fear/Greed rows:', len(fg))

# Quick preview
hist.head()

## 3. Quick inspection
Check columns, data types and missing values.

In [ ]:

hist.info()
print('\nFear/Greed info:')
fg.info()

# Show sample rows
display(hist.head(3))
display(fg.head(3))


## 4. Clean timestamps and merge
- Parse trade timestamps into `trade_date` (date only). 
- Parse fear/greed date to `fg_date` and merge on the date.

In [ ]:

# Find timestamp-like column in historical data
time_col = next((c for c in hist.columns if 'time' in c.lower() or 'date' in c.lower()), None)
print('Detected time column:', time_col)

# Parse timestamps
hist[time_col + '_parsed'] = pd.to_datetime(hist[time_col], errors='coerce')
hist['trade_date'] = hist[time_col + '_parsed'].dt.date

# Parse fear/greed date
fg_date_col = next((c for c in fg.columns if 'date' in c.lower()), None)
fg['Date_parsed'] = pd.to_datetime(fg[fg_date_col], errors='coerce')
fg['fg_date'] = fg['Date_parsed'].dt.date

# Merge
fg_small = fg[['fg_date', 'classification']].rename(columns={'classification':'Classification'})
merged = hist.merge(fg_small, left_on='trade_date', right_on='fg_date', how='left')

print('Merged rows:', len(merged))
merged[['trade_date','fg_date','Classification']].head(5)

## 5. Convert numeric columns and create indicators
Convert `Closed PnL`, `Size` to numeric and create a `win` indicator (closedPnL > 0).

In [ ]:

# Attempt to find common column names
pnl_col = next((c for c in hist.columns if 'pnl' in c.lower()), None)
size_col = next((c for c in hist.columns if 'size' in c.lower()), None)
side_col = next((c for c in hist.columns if c.lower() in ['side','direction']), None)

print('Detected columns -> pnl:', pnl_col, ', size:', size_col, ', side:', side_col)

merged['closedPnL'] = pd.to_numeric(merged.get(pnl_col, pd.Series()), errors='coerce')
merged['size'] = pd.to_numeric(merged.get(size_col, pd.Series()), errors='coerce')
merged['side'] = merged.get(side_col, pd.Series())

merged['win'] = merged['closedPnL'] > 0
merged[['closedPnL','size','side','win']].head()

## 6. Aggregate metrics by market sentiment
Compute: trade count, mean/median closedPnL, win rate, median size.

In [ ]:

grouped = merged.groupby('Classification').agg(
    trades_count = ('closedPnL','count'),
    mean_closedPnL = ('closedPnL','mean'),
    median_closedPnL = ('closedPnL','median'),
    win_rate = ('win', lambda x: 100 * x.sum()/x.count() if x.count()>0 else np.nan),
    median_size = ('size','median')
).reset_index()

grouped = grouped.sort_values('trades_count', ascending=False)
display(grouped)

# Save CSV outputs
grouped.to_csv('/mnt/data/csv_files/summary_by_sentiment.csv', index=False)
merged.groupby('trade_date').agg(trades=('closedPnL','count'), total_closedPnL=('closedPnL','sum')).reset_index().to_csv('/mnt/data/csv_files/daily_trade_summary.csv', index=False)
print('Saved summary CSVs in /mnt/data/csv_files')

## 7. Visualizations
- Trades per day
- Mean closedPnL by sentiment
- Win rate by sentiment
- PnL distribution (histogram)

Images will be saved to `/mnt/data/outputs` so you can include them in your report.

In [ ]:

# Trades per day
daily = merged.groupby('trade_date').agg(trades=('closedPnL','count')).reset_index()
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(daily['trade_date']), daily['trades'])
plt.title('Trades per day')
plt.xlabel('Date')
plt.ylabel('Number of trades')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/trades_per_day.png')
plt.close()

# Mean closedPnL by sentiment
plt.figure(figsize=(6,4))
grp = grouped.dropna(subset=['mean_closedPnL'])
plt.bar(grp['Classification'].astype(str), grp['mean_closedPnL'])
plt.title('Mean closedPnL by Market Sentiment')
plt.xlabel('Market Sentiment')
plt.ylabel('Mean closedPnL')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/mean_pnl_by_sentiment.png')
plt.close()

# Win rate by sentiment
win_rate_df = merged.groupby('Classification')['win'].mean().sort_values(ascending=False) * 100
plt.figure(figsize=(6,4))
plt.bar(win_rate_df.index.astype(str), win_rate_df.values)
plt.title('Win Rate by Market Sentiment')
plt.xlabel('Market Sentiment')
plt.ylabel('Win Rate (%)')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/win_rate_by_sentiment.png')
plt.close()

# PnL histogram (trim top 1% to focus visualization)
pnl_data = merged['closedPnL'].dropna()
trim = np.percentile(np.abs(pnl_data), 99)
pnl_trim = pnl_data[np.abs(pnl_data) <= trim]
plt.figure(figsize=(8,5))
plt.hist(pnl_trim, bins=100)
plt.title('Closed PnL Distribution (trimmed at 99th percentile)')
plt.xlabel('Closed PnL')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/pnl_distribution_histogram.png')
plt.close()

print('Saved plots to /mnt/data/outputs')

## 8. Conclusions & next steps
- Summarize findings: e.g., higher mean PnL and win rate during Extreme Greed.
- Next steps: leverage parsing/normalization, per-account analysis, build predictive models, or risk-adjusted metrics.